<a href="https://colab.research.google.com/github/shonyeajin/KCBD/blob/main/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

생성자

In [9]:
import keras
from keras import layers
import numpy as np

latent_dim=32
height=32
width=32
channels=3

generator_input=keras.Input(shape=(latent_dim,))

In [10]:
x=layers.Dense(128*16*16)(generator_input)
x=layers.LeakyReLU()(x)
x=layers.Reshape((16,16,128))(x)

x=layers.Conv2D(256,5, padding='same')(x)
x=layers.LeakyReLU()(x)

x=layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x=layers.LeakyReLU()(x)

In [11]:
x=layers.Conv2D(256, 5,padding='same')(x)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(256, 5,padding='same')(x)
x=layers.LeakyReLU()(x)

x=layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator=keras.models.Model(generator_input, x)
generator.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_4 (Dense)             (None, 32768)             1081344   
                                                                 
 leaky_re_lu_17 (LeakyReLU)  (None, 32768)             0         
                                                                 
 reshape_1 (Reshape)         (None, 16, 16, 128)       0         
                                                                 
 conv2d_16 (Conv2D)          (None, 16, 16, 256)       819456    
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 16, 16, 256)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 32, 32, 256)      1048

판별자

In [12]:
discriminator_input=layers.Input(shape=(height, width, channels))
x= layers.Conv2D(128, 3)(discriminator_input)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(128, 4, strides=2)(x)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(128, 4, strides=2)(x)
x=layers.LeakyReLU()(x)
x=layers.Conv2D(128, 4, strides=2)(x)
x=layers.LeakyReLU()(x)

x=layers.Dropout(0.4)(x)

x=layers.Dense(1, activation='sigmoid')(x)

discriminator=keras.models.Model(discriminator_input, x)
discriminator.summary()

import tensorflow as tf

discriminator_optimizer=tf.keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_20 (Conv2D)          (None, 30, 30, 128)       3584      
                                                                 
 leaky_re_lu_22 (LeakyReLU)  (None, 30, 30, 128)       0         
                                                                 
 conv2d_21 (Conv2D)          (None, 14, 14, 128)       262272    
                                                                 
 leaky_re_lu_23 (LeakyReLU)  (None, 14, 14, 128)       0         
                                                                 
 conv2d_22 (Conv2D)          (None, 6, 6, 128)         262272    
                                                                 
 leaky_re_lu_24 (LeakyReLU)  (None, 6, 6, 128)         0   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


적대적 네트워크

In [13]:
discriminator.trainable=False

gan_input=keras.Input(shape=(latent_dim,))
gan_output=discriminator(generator(gan_input))
gan=keras.models.Model(gan_input, gan_output)

gan_optimizer=tf.keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


DCGAN 훈련

In [14]:
import os
from keras.preprocessing import image

(x_train, y_train),(_,_)=keras.datasets.cifar10.load_data()

x_train=x_train[y_train.flatten()==6]

x_train=x_train.reshape(
    (x_train.shape[0],)+(height,width, channels)).astype('float32')/255.

iterations=10000
batch_size=20
save_dir='./datasets/gan_images/'
if not os.path.exists(save_dir):
  os.mkdir(save_dir)

In [17]:
start=0
for step in range(iterations):
  random_latent_vectors=np.random.normal(size=(batch_size,latent_dim))
  generated_images=generator.predict(random_latent_vectors)

  stop=start+batch_size
  real_images=x_train[start:stop]
  combined_images=np.concatenate([generated_images, real_images])

  labels=np.concatenate([np.ones((batch_size,1)),np.zeros((batch_size, 1))])

  labels+=0.05*np.random.random(labels.shape)
  d_loss=discriminator.train_on_batch(combined_images, labels)

  random_latent_vectors=np.random.normal(size=(batch_size, latent_dim))

  misleading_targets=np.zeros((batch_size,1))
  
  a_loss=gan.train_on_batch(random_latent_vectors, misleading_targets)

  start+=batch_size
  if start> len(x_train)-batch_size:
    start=0
  if step%100==0:
    gan.save_weights('gan.h5')

    print('스텝 %s에서 판별자 손실: %s'%(step, d_loss))
    print('스텝 %s에서 적대적 손실: %s'%(step, a_loss))

    img=image.array_to_img(generated_images[0]*255., scale=False)
    img.save(os.path.join(save_dir, 'generated_frog'+ str(step)+'.png'))

    img=image.array_to_img(real_images[0]*255., scale=False)
    img.save(os.path.join(save_dir, 'real_frog'+str(step)+'.png'))

ValueError: ignored